# **Lab-10-3 visdom**

## **Visdom**

![](https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=https%3A%2F%2Fk.kakaocdn.net%2Fdn%2FpDqwc%2FbtqCvy8qs8T%2FNRzxiadk9yhP165nHIXeEK%2Fimg.png)

Visdom은 라이브 데이터를 풍부하게 시각화 해주는 시각화 도구이다.  연구원과 개발자가 원격 서버에서 과학 실험을 지속적으로 할 수 있도록 도와주며, 브라우저에서 실행되며 다른 사람과 쉽게 공유할 수 있다.

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dsets

## **Import visdom**

In [2]:
import visdom
vis = visdom.Visdom()

Setting up a new session...


## **Text**

In [3]:
vis.text("Hello world!", env="main") # env는 한번에 모든 창을 끌 때 사용

'window_387aafceea7f26'

## **Image**

In [4]:
a=torch.randn(3, 200, 200) # 3은 RGB
vis.image(a)

'window_387aafe4fc98e8'

## **Images**

In [6]:
vis.images(torch.Tensor(3,3,28,28))

'window_387ab0046ac6b6'

## **example (using MNIST and CIFAR10)**

In [8]:
MNIST = dsets.MNIST(root="./MNIST_data",train=True, transform=torchvision.transforms.ToTensor(),download=True)
cifar10 = dsets.CIFAR10(root="./cifar10",train=True, transform=torchvision.transforms.ToTensor(),download=True)

Extracting ./cifar10\cifar-10-python.tar.gz to ./cifar10


### **CIFAR10**

In [10]:
data = cifar10.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([3, 32, 32])


'window_387ab3f6546fba'

### **MINIST**

In [11]:
data = MNIST.__getitem__(0)
print(data[0].shape)
vis.images(data[0],env="main")

torch.Size([1, 28, 28])


'window_387ab42fae8f16'

### **Check dataset**

In [12]:
data_loader = torch.utils.data.DataLoader(dataset = MNIST,
                                         batch_size = 32,
                                         shuffle = False)

In [13]:
for num, value in enumerate(data_loader):
    value = value[0]
    print(value.shape)
    vis.images(value)
    break

torch.Size([32, 1, 28, 28])


In [15]:
vis.close(env="main")

''

## **Line Plot**

In [16]:
Y_data = torch.randn(5)
plt = vis.line(Y=Y_data)

In [17]:
# X 값을 넣지 않으면 X축은 무조건 0~1사이의 값
X_data = torch.Tensor([1,2,3,4,5])
plt = vis.line(Y=Y_data, X=X_data)

### **Line update**

In [22]:
Y_append = torch.randn(1)
X_append = torch.Tensor([6])

vis.line(Y=Y_append, X=X_append, win=plt, update='append') # (추가할 값, 인덱스, 업데이트할 윈도우 이름, 확장방식)

'window_387ab4e20e0436'

### **multiple Line on single windows**

In [24]:
num = torch.Tensor(list(range(0, 10)))
print(num.shape)
num = num.view(-1,1)
print(num.shape)
num = torch.cat((num,num),dim=1)
print(num.shape)

plt = vis.line(Y=torch.randn(10,2), X=num)

torch.Size([10])
torch.Size([10, 1])
torch.Size([10, 2])


### **Line info**


In [25]:
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', showlegend=True))
plt = vis.line(Y=Y_data, X=X_data, opts = dict(title='Test', legend = ['1번'], showlegend=True))
plt = vis.line(Y=torch.randn(10, 2), X = num, opts=dict(title='Test', legend=['1번', '2번'], showlegend=True))

## **make function for update line**

In [27]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_value, are Tensor'''
    vis.line(X=num,
            Y=loss_value,
            win=loss_plot,
            update="append")

In [28]:
plt = vis.line(Y=torch.Tensor(1).zero_())

for i in range(500):
    loss = torch.randn(1)+i
    loss_tracker(plt, loss, torch.Tensor([i]))

In [29]:
vis.close(env="main")

''

## MNIST-CNN with Visdom

In [30]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [31]:
import visdom
vis = visdom.Visdom()
vis.close(env="main") # 기존의 창들 닫으려고

Setting up a new session...


### define loss_tracker

In [32]:
def loss_tracker(loss_plot, loss_value, num):
    '''num, loss_alue, are Tensor'''
    vis.line(X=num,
            Y=loss_value,
            win=loss_plot,
            update='append')

In [33]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [34]:
# parameters
learning_rate = 0.001
training_epochs = 15
batch_size = 32

In [35]:
# MNIST dataset
mnist_train = dsets.MNIST(root='MNIST_data/',
                          train=True,
                          transform=transforms.ToTensor(),
                          download=True)

mnist_test = dsets.MNIST(root='MNIST_data/',
                         train=False,
                         transform=transforms.ToTensor(),
                         download=True)

In [36]:
# dataset loader
data_loader = torch.utils.data.DataLoader(dataset=mnist_train,
                                          batch_size=batch_size,
                                          shuffle=True,
                                          drop_last=True)

In [38]:
# CNN Model (2 conv layers)
class CNN(torch.nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        # L1 ImgIn shape=(?, 28, 28, 1)
        #    Conv     -> (?, 28, 28, 32)
        #    Pool     -> (?, 14, 14, 32)
        self.layer1 = torch.nn.Sequential(
            torch.nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # L2 ImgIn shape=(?, 14, 14, 32)
        #    Conv      ->(?, 14, 14, 64)
        #    Pool      ->(?, 7, 7, 64)
        self.layer2 = torch.nn.Sequential(
            torch.nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2))
        # Final FC 7x7x64 inputs -> 10 outputs
        self.fc = torch.nn.Linear(7 * 7 * 64, 10, bias=True)
        torch.nn.init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(out.size(0), -1)   # Flatten them for FC
        out = self.fc(out)
        return out

In [39]:
# instantiate CNN model
model = CNN().to(device)

value = (torch.Tensor(1,1,29,29).to(device))
print( (model(value).shape ))

torch.Size([1, 10])


In [40]:
# define cost/loss & optimizer
criterion = torch.nn.CrossEntropyLoss().to(device) # Softmax is internally computed.
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

### make plot

In [44]:
loss_plt = vis.line(Y=torch.Tensor(1).zero_(),opts=dict(title='loss_tracker',legend=['loss'], showlegend=True))

### train with loss_tracker

In [ ]:
#training
total_batch = len(data_loader)
print('Learning started. It takes sometime.')
for epoch in range(training_epochs):
    avg_cost = 0
    
    for X, Y in data_loader:
        X = X.to(device)
        Y = Y.to(device)
        
        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost/total_batch

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    loss_tracker(loss_plt, torch.Tensor([avg_cost]), torch.Tensor([epoch])) # 한 epoch이 끝날때마다 loss의 평균을 넣는다.
print('Learning Finished!')

In [ ]:
# Test model and check accuracy
with torch.no_grad(): # 학습을 안한다는 의미
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device) # 한번에 집어넣기 위해 펼침
    Y_test = mnist_test.targets.to(device)

    prediction = model(X_test)
    correct_prediction = torch.argmax(prediction, 1) == Y_test
    accuracy = correct_prediction.float().mean()
    print('Accuracy:', accuracy.item())